In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, wilcoxon, shapiro
from sklearn.model_selection import GridSearchCV
import gc
import itertools
from sklearn.utils import resample
import ast
import json
import re
from sklearn.base import clone
import copy
import gower

import utils 
import model_train
from constants import *
import particle_swarm
import concurrent.futures


%load_ext autoreload
%autoreload 2

In [2]:
# List of file paths
df = pd.read_csv(path)
df

,age,workclass,fnlwgt,education,education_level,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
45218,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
45219,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
45220,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [3]:
df = utils.data_prep(df)

bounds = utils.get_nqi_bounds(df, NQIs)
levels = utils.get_cqi_levels(df, CQIs)
nqi_means = df[NQIs].mean()

In [4]:
# Define a list of ML models
models = [
    ("DT", DecisionTreeClassifier(criterion='entropy', random_state=42)),
    # ("LR", LogisticRegression(solver='lbfgs', max_iter=100, random_state=42, n_jobs=-1)),
    # ("NB", GaussianNB()),
    # ("NN", MLPClassifier(hidden_layer_sizes=(20,),solver='adam',batch_size=128,max_iter=50,early_stopping=True,random_state=42)),
    # ("RF", RandomForestClassifier(
    # n_estimators=4,
    # max_depth=5,
    # max_features='sqrt',
    # criterion='entropy',
    # n_jobs=-1,
    # warm_start=True,
    # random_state=42)),
    # ("SVM", LinearSVC(random_state=42))   
]

In [5]:
# Define the dictionary with all parameter values
parameters_dic = {'gamma': [1],
                  'k': [20],
                  'n_cluster': [20, 30, 40, 50, 60, 70, 80, 90, 100],
                  'initial_violation_threshold': [10],
                  'violation_decay_rate': [0.5],
                  'penalty_weight': [1]
                  }

# Generate all combinations of parameters using itertools.product
param_combinations = list(itertools.product(*parameters_dic.values()))

In [6]:
base_path = '/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Experiment results/V2/V2_1_A_Mean'

In [7]:
if __name__ == "__main__":
    for name, model in models:
        print(f"Training model: {name}")
        print("-----"*10)

        for param_comb in param_combinations:
            gamma, k_val, n_cluster_val, initial_violation_threshold, violation_decay_rate, penalty_weight = param_comb

            print(f"Running with k = {k_val}, n_cluster = {n_cluster_val}")

            with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
                futures = [
                    executor.submit(
                        particle_swarm.run_single_experiment, i, df, name, model, gamma, k_val,
                        n_cluster_val, initial_violation_threshold, violation_decay_rate,
                        penalty_weight, SAs, NQIs, CQIs, bounds, levels, nqi_means, base_path
                    )
                    for i in range(10)
                ]
                concurrent.futures.wait(futures)

Training model: DT
--------------------------------------------------
Running with k = 20, n_cluster = 20


/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:496: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_low, col_idx] = column_means[col_idx]
/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:497: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_high, col_idx] = column_means[col_idx]
/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:496: FutureWarning: 

Saved the best anonymized data to /Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Experiment results/V2/V2_1_A_Mean/Anonymized Data/best_anonymized_df_k20_ncluster20_DT_round8.csv


/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:496: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_low, col_idx] = column_means[col_idx]
/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:497: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_high, col_idx] = column_means[col_idx]


Saved the best anonymized data to /Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Experiment results/V2/V2_1_A_Mean/Anonymized Data/best_anonymized_df_k20_ncluster20_DT_round4.csv


...

Saved the best anonymized data to /Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Experiment results/V2/V2_1_A_Mean/Anonymized Data/best_anonymized_df_k20_ncluster100_DT_round4.csv


/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:496: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_low, col_idx] = column_means[col_idx]
/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:497: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_high, col_idx] = column_means[col_idx]


Saved the best anonymized data to /Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Experiment results/V2/V2_1_A_Mean/Anonymized Data/best_anonymized_df_k20_ncluster100_DT_round5.csv


/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:496: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_low, col_idx] = column_means[col_idx]
/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/particle_swarm.py:497: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  particle_numeric[mask_high, col_idx] = column_means[col_idx]


Saved the best anonymized data to /Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Experiment results/V2/V2_1_A_Mean/Anonymized Data/best_anonymized_df_k20_ncluster100_DT_round2.csv


In [8]:
# for name, model in models:
#     # Print the model name
#     print(f"Training model: {name}")
#     print("-----"*10)

#     for param_comb in param_combinations:

#         # Unpack parameters
#         gamma, k_val, n_cluster_val, initial_violation_threshold, violation_decay_rate, penalty_weight = param_comb

#         print(f"Running with k = {k_val}, n_cluster = {n_cluster_val}")

#         for i in range(10):
#             # Print the iteration number
#             print(f"Current Iteration: {i}")
            
#             results = particle_swarm.run_particle_swarm_experiment(
#                 df=df, 
#                 name=name,
#                 model=model, 
#                 gamma=gamma,
#                 k_val=k_val,
#                 SAs = SAs, 
#                 n_cluster_val=n_cluster_val,
#                 initial_violation_threshold=initial_violation_threshold,
#                 violation_decay_rate=violation_decay_rate,
#                 penalty_weight=penalty_weight, 
#                 NQIs=NQIs, 
#                 CQIs=CQIs, 
#                 n_population=1,
#                 maxIter=2,
#                 n_bootstrap=1,
#                 bounds=bounds, 
#                 levels=levels, 
#                 nqi_means=nqi_means, 
#                 filedirectory=os.path.join(base_path, 'Anonymized Data'),
#                 current_iter=i,
#                 aggregate_function='mean'
#             )

#             # save the results to a file
#             results_df = pd.DataFrame(results)
#             filedirectory = os.path.join(base_path, 'Tracking Info')
#             filename = f"track_info_k{k_val}_ncluster{n_cluster_val}_{name}_round{i}.csv"
#             filename = os.path.join(filedirectory, filename)
#             results_df.to_csv(filename, index=False)
